In [79]:
import cv2 #computer vision to work with webcam
import numpy as np
import os #used to work with file paths
from matplotlib import pyplot as plt #used for visualisation
import time
import mediapipe as mp #used to extract keypoints

In [80]:
#holistic
mp_holistic = mp.solutions.holistic #make predictions on body
mp_drawing = mp.solutions.drawing_utils #draw our predictions

In [81]:
def mediapipe_detection(image,model):
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    return image,results

In [82]:
def draw_landmarks(image,results):
    mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACEMESH_CONTOURS,
                              mp_drawing.DrawingSpec(color = (50,50,130),thickness = 1, circle_radius = 1),
                              mp_drawing.DrawingSpec(color = (100,100,100),thickness = 1, circle_radius = 1))
    mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS)

In [83]:
def extract_keypoints(results):
    pose = np.array([[res.x,res.y,res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x,res.y,res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    left_hand = np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    right_hand = np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose,face,left_hand,right_hand])

In [100]:
#create videos

#create media pipe model
actions = np.array(['Thank_you'])
no_sequences = 30 #number of videos to use for training
len_sequence = 30 #number of frames for each video
video = []
data_path = os.path.join('mpdata') #path for exported data

In [101]:
#create folders
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(data_path,action,str(sequence)))
        except:
            pass

In [102]:
#create videos
cap = cv2.VideoCapture(0) #to use internal camera
#create media pipe model

blank_image = 255 * np.ones(shape=[512, 512, 3], dtype=np.uint8)
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic:
    for action in actions:
        for sequence in range(no_sequences):
            for lenght in range(len_sequence):
                
                ret,frame = cap.read() #capturing a snap
                #make detections
                image, results = mediapipe_detection(frame,holistic)
                
                #draw_landmarks(image,results)
                #print(results)
                #display results
               # cv2.waitKey(100)
                if lenght == 0:
                    cv2.putText(image,'starting collecting data',(12,20), cv2.FONT_HERSHEY_SIMPLEX,1, (0,255,0),1, cv2.LINE_AA)
                    cv2.putText(image,'collecting frames for {} video_number {}'.format(action,str(sequence)),(12,10), 
                                cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255),1, cv2.LINE_AA)
                    cv2.waitKey(2000)
                else:
                    cv2.putText(image,'collecting frames for {} video_number {}'.format(action,str(sequence)),(12,150), 
                                cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255),1, cv2.LINE_AA)
                key_points = extract_keypoints(results)
                npy_path = os.path.join(data_path,action,str(sequence),str(lenght))
                np.save(npy_path,key_points)
                cv2.imshow('openCV Feed',image) #show image on screen
                video.append(image)
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
cv2.putText(blank_image,'getting ready to play the video',(12,50), cv2.FONT_HERSHEY_SIMPLEX,1, (255,255,0),1, cv2.LINE_AA)
cv2.imshow('openCV Feed',blank_image)
cv2.waitKey(4000)                    
for image in video:
    
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
    else:
        cv2.putText(image,'playing the video',(12,20), cv2.FONT_HERSHEY_SIMPLEX,1, (0,255,0),1, cv2.LINE_AA)
        cv2.waitKey(150)
        cv2.imshow('openCV Feed',image) #show image on screen
               
                
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [ ]:

for image in video:
    
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
    else:
        cv2.putText(image,'playing the video',(12,20), cv2.FONT_HERSHEY_SIMPLEX,1, (0,255,0),1, cv2.LINE_AA)
        cv2.waitKey(150)
        cv2.imshow('openCV Feed',image) #show image on screen
               
                
cap.release()
cv2.destroyAllWindows()

In [103]:
cap.release()
cv2.destroyAllWindows()

In [106]:
#pre processing the data
from sklearn.model_selection import train_test_split #used to split the data into training and evaluation
from tensorflow.keras.utils import to_categorical #used to converts data to one hat encoding

actions = np.array(['hello','how_are_you','good','Bad','Thank_you'])
label_map = {label:num for num,label in enumerate(actions)} #map the actions to enumeration

In [107]:
label_map

{'hello': 0, 'how_are_you': 1, 'good': 2, 'Bad': 3, 'Thank_you': 4}

In [108]:
#getting back the data
sequences,labels = [],[]
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for lenght in range(len_sequence):
            res = np.load(os.path.join(data_path,action,str(sequence),'{}.npy'.format(lenght)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [109]:
X = np.array(sequences)
Y = to_categorical(labels).astype(int) #one hot encoding the labels

In [110]:
#processing traing data
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size = 0.1)

In [111]:
#build and train the neural network
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

#tensor board allows to see the progress of your model while it is training
log_dir = os.path.join('logs')
call_back = TensorBoard(log_dir = log_dir)

In [147]:
model = Sequential()
model.add(LSTM(64, return_sequences = True, activation = 'relu',input_shape = (30,1662)))
model.add(LSTM(128, return_sequences = True, activation = 'relu'))
model.add(LSTM(64, return_sequences = False, activation = 'relu'))

model.add(Dense(64, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(actions.shape[0], activation = 'softmax'))

In [148]:
model.compile(optimizer ='Adam',loss='categorical_crossentropy',metrics='categorical_accuracy')

In [149]:
model.fit(X_train,y_train,epochs = 200, callbacks = [call_back])

Epoch 1/200
5/5 [==============================] - 7s 878ms/step - loss: 1.6236 - categorical_accuracy: 0.2074
Epoch 2/200
5/5 [==============================] - 0s 67ms/step - loss: 1.6114 - categorical_accuracy: 0.2222
Epoch 3/200
5/5 [==============================] - 0s 61ms/step - loss: 1.5959 - categorical_accuracy: 0.3111
Epoch 4/200
5/5 [==============================] - 0s 66ms/step - loss: 1.4881 - categorical_accuracy: 0.4074
Epoch 5/200
5/5 [==============================] - 0s 63ms/step - loss: 1.3955 - categorical_accuracy: 0.4000
Epoch 6/200
5/5 [==============================] - 0s 63ms/step - loss: 1.3080 - categorical_accuracy: 0.4222
Epoch 7/200
5/5 [==============================] - 0s 63ms/step - loss: 1.2426 - categorical_accuracy: 0.4296
Epoch 8/200
5/5 [==============================] - 0s 63ms/step - loss: 1.1005 - categorical_accuracy: 0.4444
Epoch 9/200
5/5 [==============================] - 0s 59ms/step - loss: 1.2132 - categorical_accuracy: 0.5481
Epoch 10/

5/5 [==============================] - 0s 63ms/step - loss: 1.0816 - categorical_accuracy: 0.5185
Epoch 75/200
5/5 [==============================] - 0s 59ms/step - loss: 1.0993 - categorical_accuracy: 0.4444
Epoch 76/200
5/5 [==============================] - 0s 59ms/step - loss: 1.0683 - categorical_accuracy: 0.4741
Epoch 77/200
5/5 [==============================] - 0s 59ms/step - loss: 0.9655 - categorical_accuracy: 0.5852
Epoch 78/200
5/5 [==============================] - 0s 59ms/step - loss: 1.0070 - categorical_accuracy: 0.4963
Epoch 79/200
5/5 [==============================] - 0s 63ms/step - loss: 1.0369 - categorical_accuracy: 0.5704
Epoch 80/200
5/5 [==============================] - 0s 63ms/step - loss: 0.9881 - categorical_accuracy: 0.5481
Epoch 81/200
5/5 [==============================] - 0s 67ms/step - loss: 0.9380 - categorical_accuracy: 0.5852
Epoch 82/200
5/5 [==============================] - 0s 63ms/step - loss: 0.9538 - categorical_accuracy: 0.5704
Epoch 83/200
5

5/5 [==============================] - 0s 63ms/step - loss: 0.7749 - categorical_accuracy: 0.6519
Epoch 148/200
5/5 [==============================] - 0s 63ms/step - loss: 0.7437 - categorical_accuracy: 0.6519
Epoch 149/200
5/5 [==============================] - 0s 66ms/step - loss: 0.7316 - categorical_accuracy: 0.6667
Epoch 150/200
5/5 [==============================] - 0s 63ms/step - loss: 1.1293 - categorical_accuracy: 0.5185
Epoch 151/200
5/5 [==============================] - 0s 67ms/step - loss: 1.2190 - categorical_accuracy: 0.4815
Epoch 152/200
5/5 [==============================] - 0s 71ms/step - loss: 1.4070 - categorical_accuracy: 0.3111
Epoch 153/200
5/5 [==============================] - 0s 72ms/step - loss: 1.1557 - categorical_accuracy: 0.5111
Epoch 154/200
5/5 [==============================] - 0s 67ms/step - loss: 1.1403 - categorical_accuracy: 0.5259
Epoch 155/200
5/5 [==============================] - 0s 63ms/step - loss: 1.0425 - categorical_accuracy: 0.6074
Epoch 

In [116]:
#making predictions
res_test = model.predict(X_test)
predicted_action = actions[np.argmax(res_test[1])]

In [142]:
actions[np.argmax(res_test[7])]

'good'

In [143]:
actions[np.argmax(y_test[7])]

'good'

In [150]:
!pip install pyttsx3

In [158]:
import pyttsx3

In [178]:
text_to_speech = pyttsx3.init(driverName='sapi5',debug=True)
res_test = model.predict(X_test)
text = actions[np.argmax(res_test[7])]

In [194]:
text = actions[np.argmax(res_test[7])]
print(actions[np.argmax(y_test[7])],text)


good good


In [203]:
from gtts import gTTS
import playsound

def speak(text):
    tts = gTTS(text=text, lang='en')
    filename = 'voice.mp3'
   # tts.save(filename)
    playsound.playsound(filename)

In [202]:
speak(text)


    Error 265 for command:
        open voice.mp3
    The device name is already being used as an alias by this application.  Use a unique alias.


PlaysoundException: 
    Error 265 for command:
        open voice.mp3
    The device name is already being used as an alias by this application.  Use a unique alias.

In [204]:
feature_points = []
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic:
    while cap is on:
                
        ret,frame = cap.read() #capturing a snap
        #make detections
        image, results = mediapipe_detection(frame,holistic)
        #draw_landmarks(image,results)
        key_points = extract_keypoints(results)
        feature_points.append(key_points)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            cap.release()
            cv2.destroyAllWindows()
            break
        if len(feature_points) > 29:
            #check is the image is defined
            res = model.predict(feature_points)
            if res[na.argmax(res)] > 0.5:
                #prints the action
                action = actions[na.argmax(res)]
                print(action)
                #remove all frames
                featur_points.clear()
                cv2.waitKey(200)
            else:
                #remove the first frame
                feature_points.remove(0)
            


NameError: name 'on' is not defined

In [ ]:
#play action
def play_vid(action,cam):
    sequences,labels = [],[]
    window = []
    for lenght in range(len_sequence):
        res = np.load(os.path.join(data_path,action,'{}.npy'.format(lenght)))
        window.append(res)
    sequences.append(window)
    
return sequences
    

In [ ]:
def 

In [ ]:
#speech to sign language
import SpeecRecognition as sr
#convert speech to text
r = sr.recognizer()
image = np.load(os.path.join(data_path,action,'{}.npy'.format(lenght))) #load pose image
#find the text on resources and display it to the screen
#getting back the data text = speach_text()
with sr.Microphone() as source:
    audio = r.listen(source)
    try:
        text = r.recognize_google(audio)
        if len(text)> 0:
            for image in play_vid(action):
                cv2.imshow('openCV Feed',image) #show image on screen
    except:
        cv2.imshow('openCV Feed',image) #show image on screen